In [36]:
import pandas as pd
import torch
from torch.optim import AdamW
from torch.utils.data import DataLoader, Dataset
from torch.nn import CrossEntropyLoss
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import torch.nn as nn
import matplotlib.pyplot as plt
from kobert_transformers import get_kobert_model
from kobert_transformers.tokenization_kobert import KoBertTokenizer
import numpy as np
import time
from matplotlib import rcParams
# 한글 폰트 설정
rcParams['font.family'] = 'Malgun Gothic'  # 맑은 고딕으로 설정
rcParams['axes.unicode_minus'] = False     # 음수 기호 문제 해결

In [38]:
df_bigkinds = pd.read_excel('./Text_Data/bigkinds/삼성전자_국제_20170901-20240830_전처리후.xlsx', converters={'뉴스 식별자': str})  # '뉴스 식별자'는 해당 열의 실제 이름으로 대체

In [39]:
df_bigkinds

,뉴스 식별자,일자,언론사,기고자,제목,통합 분류1,통합 분류2,통합 분류3,사건/사고 분류1,사건/사고 분류2,...,URL,분석제외 여부,제목_키워드,제목_키워드_전처리후,제목_키워드_전처리후_Okt,제목_키워드_전처리후_Hannanum,제목_키워드_전처리후_Kkma,제목_키워드_전처리후_Komoran,제목_키워드_전처리후_kiwi,제목_키워드_전처리후_Komoran_명사
0,02100101.20170901163910001,2017-09-01,매일경제,NaN,"이재용 항소심, 서울고법 형사 13부에 배당 이달중 첫 기일",사회>사건_사고,사회>교육_시험,NaN,범죄>정치>뇌물수수,NaN,...,http://news.mk.co.kr/newsRead.php?year=2017&no...,NaN,"이재용 항소심, 서울고법 형사 13부에 배당 이달중 첫 기일 이재용,항소심,13부,...",이재용 항소심 서울고법 형사 13부에 배당 이달중 첫 기일 이재용 항소심 13부 서...,이재용 항소심 서울 고법 형사 13 부 배당 달 중 첫 기일 이재용 항소심 13 부...,이재용 항소심 서울고법 형사 13부 배당 이달 중 첫 기 ㄹ 이재용 항소심 13부 ...,이재 용 항소심 서울 고법 형사 13 부 배당 이달 중 첫 기일 이재 용 항소심 1...,이재용 항소심 서울고법 형사 13 부 배당 이달 중 첫 기일 이재용 항소심 13 부...,이재용 항소심 서울고법 형사 13 배당 이달 기일 이재용 항소심 13 서울고법 형사...,이재용 항소심 서울고법 형사 부 배당 이달 중 기일 이재용 항소심 부 서울고법 형사...
1,02100101.20170901174354001,2017-09-01,매일경제,김제이,최순실게이트 청문회 불출석 11인 중 8명 `백수`,사회>사건_사고,정치>청와대,정치>국회_정당,범죄>정치>뇌물수수,NaN,...,http://news.mk.co.kr/newsRead.php?year=2017&no...,NaN,"최순실게이트 청문회 불출석 11인 중 8명 `백수` 불출석,최순실게이트,청문회,백수...",최순실게이트 청문회 불출석 11인 중 8명 백수 불출석 최순실게이트 청문회 백수 문...,최 순 실 게이트 청문회 불 출석 11 인 중 8 명 백수 불 출석 최 순 실 게이...,최순실게이트 청문회 불출석 11 ㄴ 중 8명 백수 불출석 최순실게이트 청문회 백수 ...,최 순 실 게이트 청문회 불 출석 11 인 중 8 명 백 수 불 출석 최 순 실 게...,최순 실 게이트 청문회 불 출석 11 ㄴ 중 8 명 백수 불 출석 최순 실 게이트 ...,최순실 게이트 청문회 출석 11 백수 출석 최순실 게이트 청문회 백수 문고리 안봉근...,최순 실 게이트 청문회 출석 중 명 백수 출석 최순 실 게이트 청문회 백수 문고리 ...
2,02100101.20170901181209001,2017-09-01,매일경제,정주원,`삼성 뇌물` 이재용 항소심 `신설` 형사13부 배당,사회>사건_사고,사회>여성,사회>교육_시험,범죄>정치>뇌물수수,NaN,...,http://news.mk.co.kr/newsRead.php?year=2017&no...,NaN,"`삼성 뇌물` 이재용 항소심 `신설` 형사13부 배당 삼성,뇌물,신설,이재용,항소심...",삼성 뇌물 이재용 항소심 신설 형사13부 배당 삼성 뇌물 신설 이재용 항소심 형사 ...,삼성 뇌물 이재용 항소심 신설 형사 13 부 배당 삼성 뇌물 신설 이재용 항소심 형...,삼성 뇌물 이재용 항소심 신설 형사13부 배당 삼성 뇌물 신설 이재용 항소심 형사 ...,삼성 뇌물 이재 용 항소심 신설 형사 13 부 배당 삼성 뇌물 신설 이재 용 항소심...,삼성 뇌물 이재용 항소심 신설 형사 13 부 배당 삼성 뇌물 신설 이재용 항소심 형...,삼성 뇌물 이재용 항소심 신설 형사 13 배당 삼성 뇌물 신설 이재용 항소심 형사 ...,삼성 뇌물 이재용 항소심 신설 형사 부 배당 삼성 뇌물 신설 이재용 항소심 형사 배...
3,02100101.20170901141935001,2017-09-01,매일경제,정주원,두문불출 10개월만에 `문고리 실세` 이재만 안봉근 법정 나와 청문회불출석 인정,사회>사건_사고,정치>청와대,정치>국회_정당,NaN,NaN,...,http://news.mk.co.kr/newsRead.php?year=2017&no...,NaN,두문불출 10개월만에 `문고리 실세` 이재만 안봉근 법정 나와 청문회불출석 인정 두...,두문불출 10개월만에 문고리 실세 이재만 안봉근 법정 나와 청문회불출석 인정 두문불...,두문불출 10 개월 만에 문 고리 실세 이재만 안봉 근 법정 나와 청문회 불 출석 ...,두문불출 10개월 만에 문고리 실세 이재 만 안봉근 법정 청문회불출석 인정 두문불출...,두문불출 10 개월 만 문고리 실세 이재 만 알 ㄴ 봉 근 법정 나오 청문회 불 출...,두문불출 10 개월 만 문고리 실세 이재만 안 봉 근 법정 나오 청문회 불 출석 인...,두문불출 10 개월 문고리 실세 이재만 안봉근 법정 청문회 출석 인정 두문불출 개월...,두문불출 개월 문고리 실세 이재만 안 봉 근 법정 청문회 출석 인정 두문불출 개월 ...
4,02100311.20170901153936001,2017-09-01,서울경제,장주영 기자,"청문회 불출석 혐의, 건강상태-절차적 하자로 '무죄' 주장",정치>국회_정당,사회>사건_사고,정치>청와대,NaN,NaN,...,http://www.sedaily.com/NewsView/1OKUY3ELL1,NaN,"청문회 불출석 혐의, 건강상태-절차적 하자로 '무죄' 주장 혐의,청문회,건강상태,절...",청문회 불출석 혐의 건강상태-절차적 하자로 무죄 주장 혐의 청문회 건강상태 절차적 ...,청문회 불 출석 혐의 건강 상태 - 절차 적 하자 무죄 주장 혐의 청문회 건강 상태...,청문회 불출석 혐의 건강상태 - 절차적 하자 무죄 주장 혐의 청문회 건강상태 절차적...,청문회 불 출석 혐의 건강 상태 - 절차 적 하자 무죄 주장 혐의 청문회 건강 상태...,청문회 불 출석 혐의 건강 상태 - 절차 적 하자 무죄 주장 혐의 청문회 건강 상태...,청문회 출석 혐의 건강 상태 절차 하자 주장 혐의 청문회 건강 상태 절차 하자 무죄...,청문회 출석 혐의 건강 상태 절차 하자 무죄 주장 혐의 청문회 건강 상태 절차 무죄...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4006,02100101.20240821175511003,2024-08-21,매일경제,박재영 기자(jyp8909@mk.co.kr),의족 차고 자전거 국토횡단에 감동 뭉클,사회>사회일반,NaN,NaN,사고>스포츠사고,NaN,...,http://www.mk.co.kr/article/11098054,NaN,"의족 차고 자전거 국토횡단에 감동 뭉클 의족,자전거,국토횡단,감동,뭉클,왼쪽,다리,...",의족 차고 자전거 국토횡단에 감동 뭉클 의족 자전거 국토횡단 감동 뭉클 왼쪽 다리 ...,의족 차고 자전거 국토횡단 감동 뭉클 의족 자전거 국토 횡단 감동 뭉클 왼쪽 다리 ...,의족 차 고 자전거 국토횡단 감동 뭉클 의족 자전거 국토횡단 감동 뭉클 왼쪽 다리 ...,의족 차고 자전거 국토 횡단 감동 뭉클 의족 자전거 국토 횡단 감동 뭉클 왼쪽 다리...,족 차고 자전거 국토 횡단 감동 뭉클 족 자전거 국토 횡단 감동 뭉클 왼쪽 다리 족...,의족 자전거 국토 횡단 감동 의족 자전거 국토 횡단 감동 왼쪽 다리 의족 횡단 국토...,족 차고 자전거 국토 횡단 감동 족 자전거 국토 횡단 감동 왼쪽 다리 족 횡단 국토...
4007,02100101.20240822002505001,2024-08-22,매일경제,박재영 기자(jyp8909@mk.co.kr),“자전거 국토 횡단에 650만명이 울었다”...이 선수 보자마자 ‘뭉클’,사회>사회일반,NaN,NaN,사고>스포츠사고,NaN,...,http://www.mk.co.kr/article/11098294,NaN,"“자전거 국토 횡단에 650만명이 울었다”...이 선수 보자마자 ‘뭉클’ 횡단,

In [40]:
# KoBERT 모델 및 토크나이저 로드
tokenizer = KoBertTokenizer.from_pretrained('monologg/kobert')
model = get_kobert_model()

In [41]:
# 감정 분석을 위한 회귀 레이어 추가
class KoBERTSentimentRegressor(nn.Module):
    def __init__(self, base_model, output_size=1):  # output_size를 1로 설정
        super(KoBERTSentimentRegressor, self).__init__()
        self.base_model = base_model
        self.regressor = nn.Linear(base_model.config.hidden_size, output_size)  # 연속적인 출력값을 위한 선형 레이어
    
    def forward(self, input_ids, attention_mask):
        outputs = self.base_model(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output  # [CLS] 토큰의 출력
        return self.regressor(pooled_output)

# 감정 분석 모델 생성
regressor = KoBERTSentimentRegressor(model)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
regressor.to(device)

KoBERTSentimentRegressor(
  (base_model): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(8002, 768, padding_idx=1)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [42]:
# 데이터셋 클래스 정의
class SentimentDataset(Dataset):
    def __init__(self, texts, tokenizer, max_len):
        self.texts = texts
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        encoding = self.tokenizer.encode_plus(
            text,
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        
        input_ids = encoding['input_ids'].squeeze(0)
        attention_mask = encoding['attention_mask'].squeeze(0)
        
        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask
        }

# 데이터셋 준비
MAX_LEN = 64  # 최대 토큰 길이 설정
dataset = SentimentDataset(df_bigkinds['제목_키워드_전처리후_kiwi'].values, tokenizer, MAX_LEN)
data_loader = DataLoader(dataset, batch_size=16)

In [43]:
# 감정 분석 예측 함수
def predict_sentiments(model, data_loader, device):
    model.eval()
    predictions = []
    
    with torch.no_grad():
        for batch in data_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)

            sentiment_scores = model(input_ids=input_ids, attention_mask=attention_mask)
            scores = sentiment_scores.cpu().numpy().flatten()
            # -1 ~ 1 범위로 클리핑
            scores = np.clip(scores, -1, 1)
            predictions.extend(scores)
    
    return predictions

# 감정 분석 수행
s = time.time()
predicted_scores = predict_sentiments(regressor, data_loader, device)
print("Prediction Time: ", time.time()-s)

Prediction Time:  39.355469942092896


In [44]:
# 뉴스 식별자 컬럼에서 '.' 이후의 값만 조회
df_bigkinds['뉴스 식별자'] = df_bigkinds['뉴스 식별자'].str.split('.').str[1]

In [45]:
# 데이터프레임에 결과 추가
df_bigkinds['Bert_Sentiment'] = predicted_scores
# 소수점 1자리로 반올림
df_bigkinds['Bert_Sentiment'] = df_bigkinds['Bert_Sentiment'].round(1)
df_bigkinds['Bert_Sentiment'] = df_bigkinds['Bert_Sentiment'].apply(lambda x: 0 if x == -0.0 else x)
# 결과 확인
df_bigkinds

,뉴스 식별자,일자,언론사,기고자,제목,통합 분류1,통합 분류2,통합 분류3,사건/사고 분류1,사건/사고 분류2,...,분석제외 여부,제목_키워드,제목_키워드_전처리후,제목_키워드_전처리후_Okt,제목_키워드_전처리후_Hannanum,제목_키워드_전처리후_Kkma,제목_키워드_전처리후_Komoran,제목_키워드_전처리후_kiwi,제목_키워드_전처리후_Komoran_명사,Bert_Sentiment
0,20170901163910001,2017-09-01,매일경제,NaN,"이재용 항소심, 서울고법 형사 13부에 배당 이달중 첫 기일",사회>사건_사고,사회>교육_시험,NaN,범죄>정치>뇌물수수,NaN,...,NaN,"이재용 항소심, 서울고법 형사 13부에 배당 이달중 첫 기일 이재용,항소심,13부,...",이재용 항소심 서울고법 형사 13부에 배당 이달중 첫 기일 이재용 항소심 13부 서...,이재용 항소심 서울 고법 형사 13 부 배당 달 중 첫 기일 이재용 항소심 13 부...,이재용 항소심 서울고법 형사 13부 배당 이달 중 첫 기 ㄹ 이재용 항소심 13부 ...,이재 용 항소심 서울 고법 형사 13 부 배당 이달 중 첫 기일 이재 용 항소심 1...,이재용 항소심 서울고법 형사 13 부 배당 이달 중 첫 기일 이재용 항소심 13 부...,이재용 항소심 서울고법 형사 13 배당 이달 기일 이재용 항소심 13 서울고법 형사...,이재용 항소심 서울고법 형사 부 배당 이달 중 기일 이재용 항소심 부 서울고법 형사...,-0.2
1,20170901174354001,2017-09-01,매일경제,김제이,최순실게이트 청문회 불출석 11인 중 8명 `백수`,사회>사건_사고,정치>청와대,정치>국회_정당,범죄>정치>뇌물수수,NaN,...,NaN,"최순실게이트 청문회 불출석 11인 중 8명 `백수` 불출석,최순실게이트,청문회,백수...",최순실게이트 청문회 불출석 11인 중 8명 백수 불출석 최순실게이트 청문회 백수 문...,최 순 실 게이트 청문회 불 출석 11 인 중 8 명 백수 불 출석 최 순 실 게이...,최순실게이트 청문회 불출석 11 ㄴ 중 8명 백수 불출석 최순실게이트 청문회 백수 ...,최 순 실 게이트 청문회 불 출석 11 인 중 8 명 백 수 불 출석 최 순 실 게...,최순 실 게이트 청문회 불 출석 11 ㄴ 중 8 명 백수 불 출석 최순 실 게이트 ...,최순실 게이트 청문회 출석 11 백수 출석 최순실 게이트 청문회 백수 문고리 안봉근...,최순 실 게이트 청문회 출석 중 명 백수 출석 최순 실 게이트 청문회 백수 문고리 ...,-0.2
2,20170901181209001,2017-09-01,매일경제,정주원,`삼성 뇌물` 이재용 항소심 `신설` 형사13부 배당,사회>사건_사고,사회>여성,사회>교육_시험,범죄>정치>뇌물수수,NaN,...,NaN,"`삼성 뇌물` 이재용 항소심 `신설` 형사13부 배당 삼성,뇌물,신설,이재용,항소심...",삼성 뇌물 이재용 항소심 신설 형사13부 배당 삼성 뇌물 신설 이재용 항소심 형사 ...,삼성 뇌물 이재용 항소심 신설 형사 13 부 배당 삼성 뇌물 신설 이재용 항소심 형...,삼성 뇌물 이재용 항소심 신설 형사13부 배당 삼성 뇌물 신설 이재용 항소심 형사 ...,삼성 뇌물 이재 용 항소심 신설 형사 13 부 배당 삼성 뇌물 신설 이재 용 항소심...,삼성 뇌물 이재용 항소심 신설 형사 13 부 배당 삼성 뇌물 신설 이재용 항소심 형...,삼성 뇌물 이재용 항소심 신설 형사 13 배당 삼성 뇌물 신설 이재용 항소심 형사 ...,삼성 뇌물 이재용 항소심 신설 형사 부 배당 삼성 뇌물 신설 이재용 항소심 형사 배...,-0.2
3,20170901141935001,2017-09-01,매일경제,정주원,두문불출 10개월만에 `문고리 실세` 이재만 안봉근 법정 나와 청문회불출석 인정,사회>사건_사고,정치>청와대,정치>국회_정당,NaN,NaN,...,NaN,두문불출 10개월만에 `문고리 실세` 이재만 안봉근 법정 나와 청문회불출석 인정 두...,두문불출 10개월만에 문고리 실세 이재만 안봉근 법정 나와 청문회불출석 인정 두문불...,두문불출 10 개월 만에 문 고리 실세 이재만 안봉 근 법정 나와 청문회 불 출석 ...,두문불출 10개월 만에 문고리 실세 이재 만 안봉근 법정 청문회불출석 인정 두문불출...,두문불출 10 개월 만 문고리 실세 이재 만 알 ㄴ 봉 근 법정 나오 청문회 불 출...,두문불출 10 개월 만 문고리 실세 이재만 안 봉 근 법정 나오 청문회 불 출석 인...,두문불출 10 개월 문고리 실세 이재만 안봉근 법정 청문회 출석 인정 두문불출 개월...,두문불출 개월 문고리 실세 이재만 안 봉 근 법정 청문회 출석 인정 두문불출 개월 ...,0.0
4,20170901153936001,2017-09-01,서울경제,장주영 기자,"청문회 불출석 혐의, 건강상태-절차적 하자로 '무죄' 주장",정치>국회_정당,사회>사건_사고,정치>청와대,NaN,NaN,...,NaN,"청문회 불출석 혐의, 건강상태-절차적 하자로 '무죄' 주장 혐의,청문회,건강상태,절...",청문회 불출석 혐의 건강상태-절차적 하자로 무죄 주장 혐의 청문회 건강상태 절차적 ...,청문회 불 출석 혐의 건강 상태 - 절차 적 하자 무죄 주장 혐의 청문회 건강 상태...,청문회 불출석 혐의 건강상태 - 절차적 하자 무죄 주장 혐의 청문회 건강상태 절차적...,청문회 불 출석 혐의 건강 상태 - 절차 적 하자 무죄 주장 혐의 청문회 건강 상태...,청문회 불 출석 혐의 건강 상태 - 절차 적 하자 무죄 주장 혐의 청문회 건강 상태...,청문회 출석 혐의 건강 상태 절차 하자 주장 혐의 청문회 건강 상태 절차 하자 무죄...,청문회 출석 혐의 건강 상태 절차 하자 무죄 주장 혐의 청문회 건강 상태 절차 무죄...,0.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4006,20240821175511003,2024-08-21,매일경제,박재영 기자(jyp8909@mk.co.kr),의족 차고 자전거 국토횡단에 감동 뭉클,사회>사회일반,NaN,NaN,사고>스포츠사고,NaN,...,NaN,"의족 차고 자전거 국토횡단에 감동 뭉클 의족,자전거,국토횡단,감동,뭉클,왼쪽,다리,...",의족 차고 자전거 국토횡단에 감동 뭉클 의족 자전거 국토횡단 감동 뭉클 왼쪽 다리 ...,의족 차고 자전거 국토횡단 감동 뭉클 의족 자전거 국토 횡단 감동 뭉클 왼쪽 다리 ...,의족 차 고 자전거 국토횡단 감동 뭉클 의족 자전거 국토횡단 감동 뭉클 왼쪽 다리 ...,의족 차고 자전거 국토 횡단 감동 뭉클 의족 자전거 국토 횡단 감동 뭉클 왼쪽 다리...,족 차고 자전거 국토 횡단 감동 뭉클 족 자전거 국토 횡단 감동 뭉클 왼쪽 다리 족...,의족 자전거 국토 횡단 감동 의족 자전거 국토 횡단 감동 왼쪽 다리 의족 횡단 국토...,족 차고 자전거 국토 횡단 감동 족 자전거 국토 횡단 감동 왼쪽 다리 족 횡단 국토...,0.0
4007,20240822002505001,2024-08-22,매일경제,박재영 기자(jyp8909@mk.co.kr),“자전거 국토 횡단에 650만명이 울었다”...이 선수 보자마자 ‘뭉클’,사회>사회일반,NaN,NaN,사고>스포츠사고,NaN,...,NaN,"“자전거 국토 횡단에 650만명이 울었다”...이 선수 보자마자 ‘뭉클’ 횡단,자전...",자전거 국토 횡단에 650만명이 울었다 ...이 선수 보자마자 뭉클 횡단 자전거 국...,자전거 국토 횡단 650만 명이 울었다 선수 보자마자 뭉클 횡단 자전거 국토 650...,자전거 국토 횡단 650만명 울 었다 서 ㄴ 수 보 뭉클 횡단 자전거 국토 650 ...,자전거 국토 횡단 650 만 명 울 었 다 선수 보 뭉클 횡단 자전거 국토 650 ...,자전거 국토 횡단 650 만 명 울 었 다 선수 보 뭉클 횡단 자전거 국토 650 ...,자전거 국토 횡단 650 선수 횡단 자전거 국토 650 선수 장애 사이클 선수 박찬...,자전거 국토 횡단 명 선수 횡단 자전거 국토 선수 장애 사이클 

In [46]:
df_bigkinds.columns

Index(['뉴스 식별자', '일자', '언론사', '기고자', '제목', '통합 분류1', '통합 분류2', '통합 분류3',
       '사건/사고 분류1', '사건/사고 분류2', '사건/사고 분류3', '인물', '위치', '기관', '키워드',
       '특성추출(가중치순 상위 50개)', '본문', 'URL', '분석제외 여부', '제목_키워드', '제목_키워드_전처리후',
       '제목_키워드_전처리후_Okt', '제목_키워드_전처리후_Hannanum', '제목_키워드_전처리후_Kkma',
       '제목_키워드_전처리후_Komoran', '제목_키워드_전처리후_kiwi', '제목_키워드_전처리후_Komoran_명사',
       'Bert_Sentiment'],
      dtype='object')

In [47]:
df_bigkinds.to_excel('./Text_Data/bigkinds/삼성전자_국제_20170901-20240830_전처리후_kiwi_KoBERT.xlsx', index=False)

In [48]:
df_bigkinds = df_bigkinds.groupby('일자', as_index=False)['Bert_Sentiment'].mean()
# 소수점 1자리로 반올림
df_bigkinds['Bert_Sentiment'] = df_bigkinds['Bert_Sentiment'].round(1)
df_bigkinds['Bert_Sentiment'] = df_bigkinds['Bert_Sentiment'].apply(lambda x: 0 if x == -0.0 else x)
# 결과 확인
df_bigkinds

,일자,Bert_Sentiment
0,2017-09-01,-0.1
1,2017-09-04,-0.2
2,2017-09-05,-0.1
3,2017-09-06,0.1
4,2017-09-07,0.0
...,...,...
1483,2024-08-20,-0.1
1484,2024-08-21,0.0
1485,2024-08-22,0.2
1486,2024-08-25,-0.1


In [49]:
# 감정 분석 결과 저장(일자별)
df_bigkinds.to_excel('./Text_Data/bigkinds/삼성전자_국제_20170901-20240830_전처리후_kiwi_KoBERT_daily.xlsx', index=False)

In [50]:
#KoBERT의 경우 다시 평가하는 경우 시간 소요 및 데이터가 변경되어 기존에 평가 데이터를 필터링하여 정상 데이터만 추출
import pandas as pd
# EXCEL파일 읽기
file_path = './Text_Data/bigkinds/삼성전자_국제_20170901-20240830_전처리후_kiwi_KoBERT.xlsx'
df_bigkinds = pd.read_excel(file_path, converters={'뉴스 식별자': str})  # '뉴스 식별자'는 해당 열의 실제 이름으로 대체

df_bigkinds = df_bigkinds[df_bigkinds['분석제외 여부'].isnull()]

df_bigkinds

# 그룹핑할 컬럼 리스트
group_columns = ['제목_키워드_전처리후_kiwi']

# 각 그룹에서 뉴스 식별자 컬럼의 최소값을 유지
df_bigkinds2 = df_bigkinds.groupby(group_columns, as_index=False).agg({'뉴스 식별자': 'min'})

# 필요한 컬럼만 선택하여 두 데이터프레임 병합
big_kinds_sentiment = pd.merge(
    df_bigkinds[['제목_키워드_전처리후_kiwi', '뉴스 식별자','일자','Bert_Sentiment']],
    df_bigkinds2[['제목_키워드_전처리후_kiwi', '뉴스 식별자']],
    on=['제목_키워드_전처리후_kiwi', '뉴스 식별자'],
    how='inner'
)

big_kinds_sentiment = big_kinds_sentiment[['뉴스 식별자', '일자', '제목_키워드_전처리후_kiwi', 'Bert_Sentiment']]
big_kinds_sentiment

big_kinds_sentiment.to_excel('./Text_Data/bigkinds/삼성전자_국제_20170901-20240830_전처리후_kiwi_KoBERT_filter.xlsx', index=False)

big_kinds_sentiment_avg = big_kinds_sentiment.groupby('일자', as_index=False)['Bert_Sentiment'].mean()
# 소수점 1자리로 반올림
big_kinds_sentiment_avg['Bert_Sentiment'] = big_kinds_sentiment_avg['Bert_Sentiment'].round(1)
big_kinds_sentiment_avg['Bert_Sentiment'] = big_kinds_sentiment_avg['Bert_Sentiment'].apply(lambda x: 0 if x == -0.0 else x)
# 결과 확인
big_kinds_sentiment_avg

# 감정 분석 결과 저장(일자별)
big_kinds_sentiment_avg.to_excel('./Text_Data/bigkinds/삼성전자_국제_20170901-20240830_전처리후_kiwi_KoBERT_daily_filter.xlsx', index=False)

In [51]:
big_kinds_sentiment_avg

,일자,Bert_Sentiment
0,2017-09-01,-0.1
1,2017-09-04,-0.2
2,2017-09-05,-0.1
3,2017-09-06,0.1
4,2017-09-07,0.0
...,...,...
1384,2024-08-19,-0.1
1385,2024-08-20,-0.1
1386,2024-08-21,0.0
1387,2024-08-22,0.2


In [54]:
big_kinds_sentiment

,뉴스 식별자,일자,제목_키워드_전처리후_kiwi,Bert_Sentiment
0,20170901163910001,2017-09-01,이재용 항소심 서울고법 형사 13 배당 이달 기일 이재용 항소심 13 서울고법 형사...,-0.2
1,20170901174354001,2017-09-01,최순실 게이트 청문회 출석 11 백수 출석 최순실 게이트 청문회 백수 문고리 안봉근...,-0.2
2,20170901181209001,2017-09-01,삼성 뇌물 이재용 항소심 신설 형사 13 배당 삼성 뇌물 신설 이재용 항소심 형사 ...,-0.2
3,20170901141935001,2017-09-01,두문불출 10 개월 문고리 실세 이재만 안봉근 법정 청문회 출석 인정 두문불출 개월...,0.0
4,20170901153936001,2017-09-01,청문회 출석 혐의 건강 상태 절차 하자 주장 혐의 청문회 건강 상태 절차 하자 무죄...,0.3
...,...,...,...,...
3556,20240819060124001,2024-08-19,퇴직금 퇴사 옛말 은행 은행원 퇴직금 퇴사 옛말 은행 은행원 신청 시중 은행 상반기...,-0.2
3557,20240820150930001,2024-08-20,기업 고령 임직원 구성 50 20 역전 기업 고령 임직원 구성 50 역전 임직원 연...,-0.1
3558,20240821175511003,2024-08-21,의족 자전거 국토 횡단 감동 의족 자전거 국토 횡단 감동 왼쪽 다리 의족 횡단 국토...,0.0
3559,20240822002505001,2024-08-22,자전거 국토 횡단 650 선수 횡단 자전거 국토 650 선수 장애 사이클 선수 박찬...,0.2
